# CH01.2. **Gradient Calculation**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [1]:
SEED_NUM = 2025

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [ ]:
#(1) Import libraries
import os
import random
import numpy as np
import torch

#(2) Set up options
os.environ['PYTHONHASHSEED'] = str(SEED_NUM)
random.seed(a=SEED_NUM)
np.random.seed(seed=SEED_NUM)
torch.use_deterministic_algorithms(mode=True)
torch.manual_seed(seed=SEED_NUM)

#### 00.2. **사용자정의함수 정의**

In [3]:
pass

#### 00.3. **클래스 정의**

In [4]:
pass

<b></b>

## 01. **그라디언트 계산(Gradient Calculation)**

#### 01.1. **자동 미분(Automatic Differentatiation)**

##### (1) **정의** : 복잡한 함수의 미분(특정 점의 접선의 기울기)를 자동으로 계산할 수 있게 도와주는 기능

##### (2) **동작 원리** :
##### $ \hspace{0.15cm} $ ① `requires_grad`가 설정된 텐서가 연산에 참여하게 되면, 내부적으로 연산 그래프(Computation Graph)를 구성
##### $ \hspace{0.15cm} $ ② 연산이 동작할 때마다 해당 연산에 대한 Backward 함수(연산의 역전파 정보)가 생성되어 각 텐서의 `grad_fn` 속성에 연결됨
##### $ \hspace{0.15cm} $ ③ 최종 연산에 대해 `.backward()` 메서드를 호출하면, 계산 그래프를 거슬러 올라가면서(Backprop) 기울기를 자동 연산해 `.grad` 속성에 갱신

#### 01.2. **계산 그래프(Computation Graph)**

#### <img src="../../img/01.2. Back propagation (1).png" width="50%" height="50%"></img>

##### (1) **정의** : 연산을 노드(node)와 간선(edge)로 연결해 나타낸 그래프 구조

##### (2) **자동 미분에서의 특징** :
##### $ \hspace{0.15cm} $ ① 데이터의 및 실행된 모든 연산들의 기록을 객체로 구성된 방향성 비순환 그래프(Directed Acyclic Graph)에 저장함
##### $ \hspace{0.15cm} $ ② 순전파 단계에서 요청된 연산 순서대로 결과를 계산하고, 해당 연산의 역전파 정보를 각 텐서별 `grad_fn` 속성에 갱신
##### $ \hspace{0.15cm} $ ③ 각 텐서의 `grad_fn` 속성을 통해 기울기를 계산하며, 이는 연쇄 법칙(Chain Rule)에 따라 상위(입력) 텐서로 전파됨

<b></b>

## 03. **텐서의 자동미분과 그라디언트 계산**

#### **[GRAPH]**

#### 03.1. **자동미분 준비** : `torch.tensor(···, requires_grad=True)`

In [5]:
#(1) Define `x`
x = torch.ones(size=(2, 2), dtype=torch.float64, requires_grad=True)

#(2) Print `x`
x

tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64, requires_grad=True)

In [6]:
#(3) Define `y`
y = x + 1 

#(4) Define `y`
y

tensor([[2., 2.],
        [2., 2.]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [7]:
#(5) Define `z`
z = 2 * y ** 2

#(6) Define `z`
z

tensor([[8., 8.],
        [8., 8.]], dtype=torch.float64, grad_fn=<MulBackward0>)

In [8]:
#(7) Define `obj_func`
obj_func = z.mean()

#(9) Print `obj_func`
obj_func

tensor(8., dtype=torch.float64, grad_fn=<MeanBackward0>)

#### 03.2. **역전파 시행** : `torch.tensor.grad`

In [9]:
#(1) Perform backpropagation based on `obj_func`
obj_func.backward()

#(2) Print `x.grad`
x.grad

tensor([[2., 2.],
        [2., 2.]], dtype=torch.float64)

In [10]:
#(3) Print `x`
x

tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64, requires_grad=True)

#### **(`PLUS`)** 이를 수식으로 표현하면 아래와 같음
#### $ \text{obj\_func} = \displaystyle\sum^{2}_{i=1}\displaystyle\sum^{2}_{k=1}\frac{z_{i,k}}{4} = \frac{z_{1,1} + z_{1,2} + z_{2,1} + z_{2,2}}{4} $ 
#### $ \hspace{1.45cm} = \frac{2 (y_{1,1})^{2} + 2 (y_{1,2})^{2} + 2 (y_{2,1})^{2} + 2(y_{2,2})^{2}}{4} $
#### $ \hspace{1.45cm} = \frac{2 (x_{1,1} + 1)^{2} + 2 (x_{1,2} + 1)^{2} + 2 (x_{2,1} + 1)^{2} + 2 (x_{2,2} + 1)^{2}}{4} $
#### $ \hspace{1.45cm} = \frac{(x_{1,1} + 1)^{2} + (x_{1,2} + 1)^{2} + (x_{2,1} + 1)^{2} + (x_{2,2} + 1)^{2}}{2} $
#### $ \nabla{}_{\textbf{x}} (\text{obj\_func}) = \frac{\partial{}}{\partial{}\textbf{x}} \text{obj\_func} = \begin{bmatrix} \frac{\partial{}(\text{obj\_func})}{\partial{}x_{1,1}} & \frac{\partial{}(\text{obj\_func})}{\partial{}x_{1,2}} \\ \frac{\partial{}(\text{obj\_func})}{\partial{}x_{2,1}} & \frac{\partial{}(\text{obj\_func})}{\partial{}x_{2,2}} \end{bmatrix} = \begin{bmatrix} x_{1,1}+1 & x_{1,2}+1 \\ x_{2,1}+1 & x_{2,2}+1 \end{bmatrix} \;\; $ ($ \because{} \, \textbf{x} = \begin{bmatrix} x_{1,1} & x_{1,2} \\ x_{2,1} & x_{2,2} \end{bmatrix} $)
#### $ \therefore{} \text{ if } \, x_{i,k} = 1, \;\; \frac{\partial{}}{\partial{}\textbf{x}} \text{obj\_func} = \begin{bmatrix} 2 & 2 \\ 2 & 2 \end{bmatrix} $

#### **(`PLUS`)** 실제로 pytorch에서는 미분연쇄법칙(chain rule)을 사용하여 계산함